In [1]:
!pip install selenium undetected-chromedriver bs4
!pip install --user psycopg2-binary
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup 

import re
import time
import random
import json

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import psycopg2

DB_NAME = "avito"
DB_USER = "postgres"
DB_PASSWORD = "Artem_17082003"
DB_HOST = "localhost" 
DB_PORT = "5432"       
try:
    conn = psycopg2.connect(
    dbname = DB_NAME,
    user = DB_USER,
    password = DB_PASSWORD,
    host = DB_HOST,
    port = DB_PORT)
    print('Подключение установлено')
except Exception as e:
    print(f'ошибка подключения {e}')

Подключение установлено


In [14]:
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS url_to_ads(
    id SERIAL PRIMARY KEY,
    url TEXT NOT NULL 
    );
""")

conn.commit()
cursor.close()
print('Таблица создана')

Таблица создана


In [15]:
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS info(
    id SERIAL PRIMARY KEY,
    url TEXT NOT NULL,
    title TEXT NOT NULL,
    price TEXT NOT NULL,
    photos JSONB,
    about_flat_jsonb JSONB,
    address TEXT NOT NULL,
    description TEXT NOT NULL
    );
""")

conn.commit()
cursor.close()
print('Таблица создана')

Таблица создана


In [16]:
class UrlCollector:
    def __init__(self, url, db_config):
        chrome_version = 135
        options = uc.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.6998.89 Safari/537.36')
        
        self.browser = uc.Chrome(options=options)
        self.url = url
        
        self.db_config = db_config 
        self.conn = psycopg2.connect(**db_config) 
        self.cursor = self.conn.cursor()
        
        
        
    def price_range(self, start_val, end_val):
        try:
            self.browser.get(self.url)
            wait = WebDriverWait(self.browser, 15)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

  
            start_price = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//input[@data-marker="price-from/input"]')))
            start_price.clear()
            for digit in start_val:
                start_price.send_keys(digit)
                time.sleep(0.3)  

 
            finish_price = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//input[@data-marker="price-to/input"]')))
            finish_price.clear()
            for digit in end_val:
                finish_price.send_keys(digit)
                time.sleep(0.3)  
            
            print(f"Установлен диапазон: {start_val} - {end_val} руб.")
            time.sleep(2)


            show_ads = wait.until(EC.element_to_be_clickable(
                (By.XPATH, '//button[@data-marker="search-filters/submit-button"]')))
            show_ads.click()
            time.sleep(3)
            return True
            
        except Exception as e:
            print(f"Ошибка в price_range: {str(e)}")
            return False

        
    def get_url_to_ads(self):
        try:
            WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '[data-marker="item-title"]')))

            soup = BeautifulSoup(self.browser.page_source, 'html.parser')
            links = soup.find_all('a', {'data-marker': "item-title"})
            urls = [link.get('href') for link in links if link.get('href')]

            full_urls = [url if url.startswith('http') else f"https://www.avito.ru{url}" for url in urls]
            return full_urls

        except Exception as e:
            print(f"Ошибка в get_url_to_ads: {str(e)}")
            return []
    
    
    def paginate(self):
        try:
            page = 2
            while page <= 100:
                try:
                    time.sleep(random.uniform(2, 4))
                    next_page_btn = WebDriverWait(self.browser, 15).until(
                        EC.presence_of_element_located(
                            (By.XPATH, f'//a[@data-marker="pagination-button/page({page})"]')
                        )
                    )

                    next_page_url = next_page_btn.get_attribute("href")
                    if not next_page_url:
                        print(f"\nURL страницы {page} не найден")
                        break

                    if not next_page_url.startswith('http'):
                        next_page_url = f"https://www.avito.ru{next_page_url}"

                    print(f"\nПереход на страницу {page}")
                    self.browser.get(next_page_url)

                    WebDriverWait(self.browser, 15).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-marker="item-title"]')))

                    urls = self.get_url_to_ads()
                    self.save_urls(urls)

                    page += 1

                except TimeoutException:
                    print(f"\nНе удалось найти страницу {page}. Пагинация завершена.")
                    break
                except Exception as e:
                    print(f"\nОшибка при обработке страницы {page}: {str(e)}")
                    break

        except Exception as e:
            print(f"Ошибка в paginate: {str(e)}")
    
    
    def process_all_ranges(self):
        prices = [
                    '1000', '450000', '450001', '600000', '600001', '850000', '850001', '1050000', '1050001', '1250000', '1250001', '1400000', '1400001', '1450000', '1450001', '1500000', '1500001', '1550000', '1550001', '1600000', '1600001', '1650000', '1650001', '1700000', '1700001', '1750000', '1750001', '1800000', '1800001', '1850000', '1850001', '1900000', '1900001', '1950000', '1950001', '2000000', '2000001', '2050000', '2050001', '2100000', '2100001', '2150000', '2150001', '2200000', '2200001', '2250000', '2250001', '2300000', '2300001', '2350000', '2350001', '2400000', '2400001', '2450000', '2450001', '2500000', '2500001', '2550000', '2550001', '2600000', '2600001', '2650000', '2650001', '2700000', '2700001', '2750000', '2750001', '2800000', '2800001', '2850000', '2850001', '2900000', '2900001', '2950000', '2950001', '3000000', '3000001', '3050000', '3050001', '3100000', '3100001', '3150000', '3150001', '3200000', '3200001', '3250000', '3250001', '3300000', '3300001', '3350000', '3350001', '3400000', '3400001', '3450000', '3450001', '3500000', '3500001', '3550000', '3550001', '3600000', '3600001', '3650000', '3650001', '3700000', '3700001', '3750000', '3750001', '3800000', '3800001', '3850000', '3850001', '3900000', '3900001', '3950000', '3950001', '4000000', '4000001', '4050000', '4050001', '4100000', '4100001', '4150000', '4150001', '4200000', '4200001', '4250000', '4250001', '4300000', '4300001', '4350000', '4350001', '4400000', '4400001', '4450000', '4450001', '4500000', '4500001', '4550000', '4550001', '4600000', '4600001', '4650000', '4650001', '4700000', '4700001', '4750000', '4750001', '4800000', '4800001', '4850000', '4850001', '4900000', '4900001', '4950000', '4950001', '5000000', '5000001', '5050000', '5050001', '5100000', '5100001', '5150000', '5150001', '5200000', '5200001', '5250000', '5250001', '5300000', '5300001', '5350000', '5350001', '5400000', '5400001', '5450000', '5450001', '5500000', '5500001', '5550000', '5550001', '5600000', '5600001', '5650000', '5650001', '5700000', '5700001', '5750000', '5750001', '5800000', '5800001', '5850000', '5850001', '5900000', '5900001', '5950000', '5950001', '6000000', '6000001', '6050000', '6050001', '6100000', '6100001', '6150000', '6150001', '6200000', '6200001', '6250000', '6250001', '6300000', '6300001', '6350000', '6350001', '6400000', '6400001', '6450000', '6450001', '6500000', '6500001', '6550000', '6550001', '6600000', '6600001', '6650000', '6650001', '6700000', '6700001', '6750000', '6750001', '6800000', '6800001', '6850000', '6850001', '6900000', '6900001', '6950000', '6950001', '7000000', '7000001', '7050000', '7050001', '7100000', '7100001', '7150000', '7150001', '7200000', '7200001', '7250000', '7250001', '7300000', '7300001', '7350000', '7350001', '7400000', '7400001', '7450000', '7450001', '7500000', '7500001', '7550000', '7550001', '7600000', '7600001', '7650000', '7650001', '7700000', '7700001', '7750000', '7750001', '7800000', '7800001', '7850000', '7850001', '7900000', '7900001', '7950000', '7950001', '8000000', '8000001', '8050000', '8050001', '8100000', '8100001', '8150000', '8150001', '8200000', '8200001', '8250000', '8250001', '8300000', '8300001', '8350000', '8350001', '8400000', '8400001', '8450000', '8450001', '8500000', '8500001', '8550000', '8550001', '8600000', '8600001', '8650000', '8650001', '8700000', '8700001', '8750000', '8750001', '8800000', '8800001', '8850000', '8850001', '8900000', '8900001', '8950000', '8950001', '9000000', '9000001', '9050000', '9050001', '9100000', '9100001', '9150000', '9150001', '9200000', '9200001', '9250000', '9250001', '9300000', '9300001', '9350000', '9350001', '9400000', '9400001', '9450000', '9450001', '9500000', '9500001', '9550000', '9550001', '9600000', '9600001', '9650000', '9650001', '9700000', '9700001', '9750000', '9750001', '9800000', '9800001', '9850000', '9850001', '9900000', '9900001', '9950000', '9950001', '10000000', '10000001', '10050000', '10050001', '10100000', '10100001', '10150000', '10150001', '10200000', '10200001', '10250000', '10250001', '10300000', '10300001', '10350000', '10350001', '10400000', '10400001', '10450000', '10450001', '10500000', '10500001', '10550000', '10550001', '10600000', '10600001', '10650000', '10650001', '10700000', '10700001', '10750000', '10750001', '10800000', '10800001', '10850000', '10850001', '10900000', '10900001', '10950000', '10950001', '11000000', '11000001', '11050000', '11050001', '11100000', '11100001', '11150000', '11150001', '11200000', '11200001', '11250000', '11250001', '11300000', '11300001', '11350000', '11350001', '11400000', '11400001', '11450000', '11450001', '11500000', '11500001', '11550000', '11550001', '11600000', '11600001', '11650000', '11650001', '11700000', '11700001', '11750000', '11750001', '11800000', '11800001', '11850000', '11850001', '11900000', '11900001', '11950000', '11950001', '12000000', '12000001', '12050000', '12050001', '12100000', '12100001', '12150000', '12150001', '12200000', '12200001', '12250000', '12250001', '12300000', '12300001', '12350000', '12350001', '12400000', '12400001', '12450000', '12450001', '12500000', '12500001', '12550000', '12550001', '12600000', '12600001', '12650000', '12650001', '12700000', '12700001', '12750000', '12750001', '12800000', '12800001', '12850000', '12850001', '12900000', '12900001', '12950000', '12950001', '13000000', '13000001', '13050000', '13050001', '13100000', '13100001', '13150000', '13150001', '13200000', '13200001', '13250000', '13250001', '13300000', '13300001', '13350000', '13350001', '13400000', '13400001', '13450000', '13450001', '13500000', '13500001', '13550000', '13550001', '13600000', '13600001', '13650000', '13650001', '13700000', '13700001', '13750000', '13750001', '13800000', '13800001', '13850000', '13850001', '13900000', '13900001', '13950000', '13950001', '14000000', '14000001', '14050000', '14050001', '14100000', '14100001', '14150000', '14150001', '14200000', '14200001', '14250000', '14250001', '14300000', '14300001', '14350000', '14350001', '14400000', '14400001', '14450000', '14450001', '14500000', '14500001', '14550000', '14550001', '14600000', '14600001', '14650000', '14650001', '14700000', '14700001', '14750000', '14750001', '14800000', '14800001', '14850000', '14850001', '14900000', '14900001', '14950000', '14950001', '15000000', '15000001', '15050000', '15050001', '15100000', '15100001', '15150000', '15150001', '15200000', '15200001', '15250000', '15250001', '15300000', '15300001', '15350000', '15350001', '15400000', '15400001', '15450000', '15450001', '15500000', '15500001', '15550000', '15550001', '15600000', '15600001', '15650000', '15650001', '15700000', '15700001', '15750000', '15750001', '15800000', '15800001', '15850000', '15850001', '15900000', '15900001', '15950000', '15950001', '16000000', '16000001', '16050000', '16050001', '16100000', '16100001', '16150000', '16150001', '16200000', '16200001', '16250000', '16250001', '16300000', '16300001', '16350000', '16350001', '16400000', '16400001', '16450000', '16450001', '16500000', '16500001', '16550000', '16550001', '16600000', '16600001', '16650000', '16650001', '16700000', '16700001', '16750000', '16750001', '16800000', '16800001', '16850000', '16850001', '16900000', '16900001', '16950000', '16950001', '17000000', '17000001', '17050000', '17050001', '17100000', '17100001', '17150000', '17150001', '17200000', '17200001', '17250000', '17250001', '17300000', '17300001', '17350000', '17350001', '17400000', '17400001', '17450000', '17450001', '17500000', '17500001', '17550000', '17550001', '17600000', '17600001', '17650000', '17650001', '17700000', '17700001', '17750000', '17750001', '17800000', '17800001', '17850000', '17850001', '17900000', '17900001', '17950000', '17950001', '18000000', '18000001', '18050000', '18050001', '18100000', '18100001', '18150000', '18150001', '18200000', '18200001', '18250000', '18250001', '18300000', '18300001', '18350000', '18350001', '18400000', '18400001', '18450000', '18450001', '18500000', '18500001', '18550000', '18550001', '18600000', '18600001', '18650000', '18650001', '18700000', '18700001', '18750000', '18750001', '18800000', '18800001', '18850000', '18850001', '18900000', '18900001', '18950000', '18950001', '19000000', '19000001', '19050000', '19050001', '19100000', '19100001', '19150000', '19150001', '19200000', '19200001', '19250000', '19250001', '19300000', '19300001', '19350000', '19350001', '19400000', '19400001', '19450000', '19450001', '19500000', '19500001', '19550000', '19550001', '19600000', '19600001', '19650000', '19650001', '19700000', '19700001', '19750000', '19750001', '19800000', '19800001', '19850000', '19850001', '19900000', '19900001', '19950000', '19950001', '20000000', '20000001', '20050000', '20050001', '20100000', '20100001', '20150000', '20150001', '20200000', '20200001', '20250000', '20250001', '20300000', '20300001', '20350000', '20350001', '20400000', '20400001', '20450000', '20450001', '20500000', '20500001', '20550000', '20550001', '20600000', '20600001', '20650000', '20650001', '20700000', '20700001', '20750000', '20750001', '20800000', '20800001', '20850000', '20850001', '20900000', '20900001', '20950000', '20950001', '21000000', '21000001', '21050000', '21050001', '21100000', '21100001', '21150000', '21150001', '21200000', '21200001', '21250000', '21250001', '21300000', '21300001', '21350000', '21350001', '21400000', '21400001', '21450000', '21450001', '21500000', '21500001', '21550000', '21550001', '21600000', '21600001', '21650000', '21650001', '21700000', '21700001', '21750000', '21750001', '21800000', '21800001', '21850000', '21850001', '21900000', '21900001', '21950000', '21950001', '22000000', '22000001', '22050000', '22050001', '22100000', '22100001', '22150000', '22150001', '22200000', '22200001', '22250000', '22250001', '22300000', '22300001', '22350000', '22350001', '22400000', '22400001', '22450000', '22450001', '22500000', '22500001', '22550000', '22550001', '22600000', '22600001', '22650000', '22650001',
                    '22700000', '22700001', '22750000', '22750001', '22800000', '22800001', '22850000', '22850001', '22900000', '22900001', '22950000', '22950001', '23000000', '23000001', '23050000', '23050001', '23100000', '23100001', '23150000', '23150001', '23200000', '23200001', '23250000', '23250001', '23300000', '23300001', '23350000', '23350001', '23400000', '23400001', '23450000', '23450001', '23500000', '23500001', '23550000', '23550001', '23600000', '23600001', '23650000', '23650001', '23700000', '23700001', '23750000', '23750001', '23800000', '23800001', '23850000', '23850001', '23900000', '23900001', '23950000', '23950001', '24000000', '24000001', '24050000', '24050001', '24100000', '24100001', '24150000', '24150001', '24200000', '24200001', '24250000', '24250001', '24300000', '24300001', '24350000', '24350001', '24400000', '24400001', '24450000', '24450001', '24500000', '24500001', '24550000', '24550001', '24600000', '24600001', '24650000', '24650001', '24700000', '24700001', '24750000', '24750001', '24800000', '24800001', '24850000', '24850001', '24900000', '24900001', '24950000', '24950001', '25000000', '25000001', '25050000', '25050001', '25100000', '25100001', '25150000', '25150001', '25200000', '25200001', '25250000', '25250001', '25300000', '25300001', '25350000', '25350001', '25400000', '25400001', '25450000', '25450001', '25500000', '25500001', '25550000', '25550001', '25600000', '25600001', '25650000', '25650001', '25700000', '25700001', '25750000', '25750001', '25800000', '25800001', '25850000', '25850001', '25900000', '25900001', '25950000', '25950001', '26000000', '26000001', '26050000', '26050001', '26100000', '26100001', '26150000', '26150001', '26200000', '26200001', '26250000', '26250001', '26300000', '26300001', '26350000', '26350001', '26400000', '26400001', '26450000', '26450001', '26500000', '26500001', '26550000', '26550001', '26600000', '26600001', '26650000', '26650001', '26700000', '26700001', '26750000', '26750001', '26800000', '26800001', '26850000', '26850001', '26900000', '26900001', '26950000', '26950001', '27000000', '27000001', '27050000', '27050001', '27100000', '27100001', '27150000', '27150001', '27200000', '27200001', '27250000', '27250001', '27300000', '27300001', '27350000', '27350001', '27400000', '27400001', '27450000', '27450001', '27500000', '27500001', '27550000', '27550001', '27600000', '27600001', '27650000', '27650001', '27700000', '27700001', '27750000', '27750001', '27800000', '27800001', '27850000', '27850001', '27900000', '27900001', '27950000', '27950001', '28000000', '28000001', '28050000', '28050001', '28100000', '28100001', '28150000', '28150001', '28200000', '28200001', '28250000', '28250001', '28300000', '28300001', '28350000', '28350001', '28400000', '28400001', '28450000', '28450001', '28500000', '28500001', '28550000', '28550001', '28600000', '28600001', '28650000', '28650001', '28700000', '28700001', '28750000', '28750001', '28800000', '28800001', '28850000', '28850001', '28900000', '28900001', '28950000', '28950001', '29000000', '29000001', '29050000', '29050001', '29100000', '29100001', '29150000', '29150001', '29200000', '29200001', '29250000', '29250001', '29300000', '29300001', '29350000', '29350001', '29400000', '29400001', '29450000', '29450001', '29500000', '29500001', '29550000', '29550001', '29600000', '29600001', '29650000', '29650001', '29700000', '29700001', '29750000', '29750001', '29800000', '29800001', '29850000', '29850001', '29900000', '29900001', '29950000', '29950001', '30000000', '30000001', '30050000', '30050001', '30100000', '30100001', '30150000', '30150001', '30200000', '30200001', '30250000', '30250001', '30300000', '30300001', '30350000', '30350001', '30400000', '30400001', '30450000', '30450001', '30500000', '30500001', '30550000', '30550001', '30600000', '30600001', '30650000', '30650001', '30700000', '30700001', '30750000', '30750001', '30800000', '30800001', '30850000', '30850001', '30900000', '30900001', '30950000', '30950001', '31000000', '31000001', '31050000', '31050001', '31100000', '31100001', '31150000', '31150001', '31200000', '31200001', '31250000', '31250001', '31300000', '31300001', '31350000', '31350001', '31400000', '31400001', '31450000', '31450001', '31500000', '31500001', '31550000', '31550001', '31600000', '31600001', '31650000', '31650001', '31700000', '31700001', '31750000', '31750001', '31800000', '31800001', '31850000', '31850001', '31900000', '31900001', '31950000', '31950001', '32000000', '32000001', '32050000', '32050001', '32100000', '32100001', '32150000', '32150001', '32200000', '32200001', '32250000', '32250001', '32300000', '32300001', '32350000', '32350001', '32400000', '32400001', '32450000', '32450001', '32500000', '32500001', '32550000', '32550001', '32600000', '32600001', '32650000', '32650001', '32700000', '32700001', '32750000', '32750001', '32800000', '32800001', '32850000', '32850001', '32900000', '32900001', '32950000', '32950001', '33000000', '33000001', '33050000', '33050001', '33100000', '33100001', '33150000', '33150001', '33200000', '33200001', '33250000', '33250001', '33300000', '33300001', '33350000', '33350001', '33400000', '33400001', '33450000', '33450001', '33500000', '33500001', '33550000', '33550001', '33600000', '33600001', '33650000', '33650001', '33700000', '33700001', '33750000', '33750001', '33800000', '33800001', '33850000', '33850001', '33900000', '33900001', '33950000', '33950001', '34000000', '34000001', '34050000', '34050001', '34100000', '34100001', '34150000', '34150001', '34200000', '34200001', '34250000', '34250001', '34300000', '34300001', '34350000', '34350001', '34400000', '34400001', '34450000', '34450001', '34500000', '34500001', '34550000', '34550001', '34600000', '34600001', '34650000', '34650001', '34700000', '34700001', '34750000', '34750001', '34800000', '34800001', '34850000', '34850001', '34900000', '34900001', '34950000', '34950001', '35000000', '35000001', '35050000', '35050001', '35100000', '35100001', '35150000', '35150001', '35200000', '35200001', '35250000', '35250001', '35300000', '35300001', '35350000', '35350001', '35400000', '35400001', '35450000', '35450001', '35500000', '35500001', '35550000', '35550001', '35600000', '35600001', '35650000', '35650001', '35700000', '35700001', '35750000', '35750001', '35800000', '35800001', '35850000', '35850001', '35900000', '35900001', '35950000', '35950001', '36000000', '36000001', '36050000', '36050001', '36100000', '36100001', '36150000', '36150001', '36200000', '36200001', '36250000', '36250001', '36300000', '36300001', '36350000', '36350001', '36400000', '36400001', '36450000', '36450001', '36500000', '36500001', '36550000', '36550001', '36600000', '36600001', '36650000', '36650001', '36700000', '36700001', '36750000', '36750001', '36800000', '36800001', '36850000', '36850001', '36900000', '36900001', '36950000', '36950001', '37000000', '37000001', '37050000', '37050001', '37100000', '37100001', '37150000', '37150001', '37200000', '37200001', '37250000', '37250001', '37300000', '37300001', '37350000', '37350001', '37400000', '37400001', '37450000', '37450001', '37500000', '37500001', '37550000', '37550001', '37600000', '37600001', '37650000', '37650001', '37700000', '37700001', '37750000', '37750001', '37800000', '37800001', '37850000', '37850001', '37900000', '37900001', '37950000', '37950001', '38000000', '38000001', '38050000', '38050001', '38100000', '38100001', '38150000', '38150001', '38200000', '38200001', '38250000', '38250001', '38300000', '38300001', '38350000', '38350001', '38400000', '38400001', '38450000', '38450001', '38500000', '38500001', '38550000', '38550001', '38600000', '38600001', '38650000', '38650001', '38700000', '38700001', '38750000', '38750001', '38800000', '38800001', '38850000', '38850001', '38900000', '38900001', '38950000', '38950001', '39000000', '39000001', '39050000', '39050001', '39100000', '39100001', '39150000', '39150001', '39200000', '39200001', '39250000', '39250001', '39300000', '39300001', '39350000', '39350001', '39400000', '39400001', '39450000', '39450001', '39500000', '39500001', '39550000', '39550001', '39600000', '39600001', '39650000', '39650001', '39700000', '39700001', '39750000', '39750001', '39800000', '39800001', '39850000', '39850001', '39900000', '39900001', '39950000', '39950001', '40000000', '40000001', '40050000', '40050001', '40100000', '40100001', '40150000', '40150001', '40200000', '40200001', '40250000', '40250001', '40300000', '40300001', '40350000', '40350001', '40400000', '40400001', '40450000', '40450001', '40500000', '40500001', '40550000', '40550001', '40600000', '40600001', '40650000', '40650001', '40700000', '40700001', '40750000', '40750001', '40800000', '40800001', '40850000', '40850001', '40900000', '40900001', '40950000', '40950001', '41000000', '41000001', '41050000', '41050001', '41100000', '41100001', '41150000', '41150001', '41200000', '41200001', '41250000', '41250001', '41300000', '41300001', '41350000', '41350001', '41400000', '41400001', '41450000', '41450001', '41500000', '41500001', '41550000', '41550001', '41600000', '41600001', '41650000', '41650001', '41700000', '41700001', '41750000', '41750001', '41800000', '41800001', '41850000', '41850001', '41900000', '41900001', '41950000', '41950001', '42000000', '42000001', '42050000', '42050001', '42100000', '42100001', '42150000', '42150001', '42200000', '42200001', '42250000', '42250001', '42300000', '42300001', '42350000', '42350001', '42400000', '42400001', '42450000', '42450001', '42500000', '42500001', '42550000', '42550001', '42600000', '42600001', '42650000', '42650001', '42700000', '42700001', '42750000', '42750001', '42800000', '42800001', '42850000', '42850001', '42900000', '42900001', '42950000', '42950001', '43000000', '43000001', '43050000', '43050001', '43100000', '43100001', '43150000', '43150001', '43200000', '43200001', '43250000', '43250001', '43300000', '43300001', '43350000', '43350001', '43400000', '43400001', '43450000', '43450001',
                    '43500000', '43500001', '43550000', '43550001', '43600000', '43600001', '43650000', '43650001', '43700000', '43700001', '43750000', '43750001', '43800000', '43800001', '43850000', '43850001', '43900000', '43900001', '43950000', '43950001', '44000000', '44000001', '44050000', '44050001', '44100000', '44100001', '44150000', '44150001', '44200000', '44200001', '44250000', '44250001', '44300000', '44300001', '44350000', '44350001', '44400000', '44400001', '44450000', '44450001', '44500000', '44500001', '44550000', '44550001', '44600000', '44600001', '44650000', '44650001', '44700000', '44700001', '44750000', '44750001', '44800000', '44800001', '44850000', '44850001', '44900000', '44900001', '44950000', '44950001', '45000000', '45000001', '45050000', '45050001', '45100000', '45100001', '45150000', '45150001', '45200000', '45200001', '45250000', '45250001', '45300000', '45300001', '45350000', '45350001', '45400000', '45400001', '45450000', '45450001', '45500000', '45500001', '45550000', '45550001', '45600000', '45600001', '45650000', '45650001', '45700000', '45700001', '45750000', '45750001', '45800000', '45800001', '45850000', '45850001', '45900000', '45900001', '45950000', '45950001', '46000000', '46000001', '46050000', '46050001', '46100000', '46100001', '46150000', '46150001', '46200000', '46200001', '46250000', '46250001', '46300000', '46300001', '46350000', '46350001', '46400000', '46400001', '46450000', '46450001', '46500000', '46500001', '46550000', '46550001', '46600000', '46600001', '46650000', '46650001', '46700000', '46700001', '46750000', '46750001', '46800000', '46800001', '46850000', '46850001', '46900000', '46900001', '46950000', '46950001', '47000000', '47000001', '47050000', '47050001', '47100000', '47100001', '47150000', '47150001', '47200000', '47200001', '47250000', '47250001', '47300000', '47300001', '47350000', '47350001', '47400000', '47400001', '47450000', '47450001', '47500000', '47500001', '47550000', '47550001', '47600000', '47600001', '47650000', '47650001', '47700000', '47700001', '47750000', '47750001', '47800000', '47800001', '47850000', '47850001', '47900000', '47900001', '47950000', '47950001', '48000000', '48000001', '48050000', '48050001', '48100000', '48100001', '48150000', '48150001', '48200000', '48200001', '48250000', '48250001', '48300000', '48300001', '48350000', '48350001', '48400000', '48400001', '48450000', '48450001', '48500000', '48500001', '48550000', '48550001', '48600000', '48600001', '48650000', '48650001', '48700000', '48700001', '48750000', '48750001', '48800000', '48800001', '48850000', '48850001', '48900000', '48900001', '48950000', '48950001', '49000000', '49000001', '49050000', '49050001', '49100000', '49100001', '49150000', '49150001', '49200000', '49200001', '49250000', '49250001', '49300000', '49300001', '49350000', '49350001', '49400000', '49400001', '49450000', '49450001', '49500000', '49500001', '49550000', '49550001', '49600000', '49600001', '49650000', '49650001', '49700000', '49700001', '49750000', '49750001', '49800000', '49800001', '49850000', '49850001', '49900000', '49900001', '49950000', '49950001', '50000000', '50000001', '50050000', '50050001', '50100000', '50100001', '50150000', '50150001', '50200000', '50200001', '50250000', '50250001', '50300000', '50300001', '50350000', '50350001', '50400000', '50400001', '50450000', '50450001', '50500000', '50500001', '50550000', '50550001', '50600000', '50600001', '50650000', '50650001', '50700000', '50700001', '50750000', '50750001', '50800000', '50800001', '50850000', '50850001', '50900000', '50900001', '50950000', '50950001', '51000000', '51000001', '51050000', '51050001', '51100000', '51100001', '51150000', '51150001', '51200000', '51200001', '51250000', '51250001', '51300000', '51300001', '51350000', '51350001', '51400000', '51400001', '51450000'
                 ]
        for i in range(0, len(prices)-1, 2):
            start_val = prices[i]
            end_val = prices[i+1]
            
            print(f"\n=== Обработка диапазона {start_val}-{end_val} ===")
            
            if not self.price_range(start_val, end_val):
                print(f"Пропускаем диапазон {start_val}-{end_val} из-за ошибки")
                continue
                
            urls = self.get_url_to_ads()
            self.save_urls(urls)
            self.paginate()
    
    
    def save_urls(self, urls):
        for idx, full_url in enumerate(urls, 1):
            print(f"{idx}. {full_url}")
            try:
                self.cursor.execute("INSERT INTO url_to_ads (url) VALUES (%s)", (full_url,))
            except Exception as e:
                print(f"Ошибка при сохранении URL {full_url}: {e}")
        self.conn.commit()


In [17]:
class AdParser:
    def __init__(self, url, db_config):
        chrome_version = 135
        options = uc.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.6998.89 Safari/537.36')
        
        self.browser = uc.Chrome(options=options)
        self.url = url
        
        self.db_config = db_config 
        self.conn = psycopg2.connect(**db_config) 
        self.cursor = self.conn.cursor()
        
        
    def parse_single_ad(self, url):
        try:
            self.browser.get(url)
            
            WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            time.sleep(2)
            
            soup = BeautifulSoup(self.browser.page_source, 'html.parser')
            
            
            title_tag = soup.find('h1', {'data-marker': 'item-view/title-info'})
            if title_tag:
                title = title_tag.get_text().replace('\xa0', ' ').strip()
            else:
                title = "Нет заголовка"
            print(title)
            
                
            price_tag = soup.find('span', {'itemprop': 'price'})
            if price_tag and price_tag.has_attr('content'):
                price = price_tag['content']
            else:
                price = "Нет цены"
            print(price)    
            
                
            image_tags = soup.find_all('li', {'data-marker': 'image-preview/item'})
            image_urls = []
            for el in image_tags:
                img_tag = el.find('img')
                if img_tag and img_tag.get('src'):
                    image_urls.append(img_tag['src'])
            images_jsonb = json.dumps(image_urls, ensure_ascii=False)
            print(images_jsonb)
            
            
            about_flat = {}
            about_flat_items = soup.find_all('li', class_='params-paramsList__item-_2Y2O')
            for item in about_flat_items:
                spans = item.find_all('span')
                if len(spans) >= 1:
                    key = spans[0].get_text(strip=True).replace(':', '')
                    value = item.get_text(strip=True).replace(key, '').strip(': \xa0')
                    about_flat[key] = value
            about_flat_jsonb = json.dumps(about_flat, ensure_ascii=False)
            print(about_flat_jsonb)    
                    
            
            address_tag = soup.find('span', class_='style-item-address__string-wt61A')
            if address_tag:
                address = address_tag.get_text(strip=True)
            else:
                address = 'Адрес не найден'
            print(address)
            

            description_tag = soup.find('div', {'data-marker': 'item-view/item-description'})
            if description_tag: 
                description = description_tag.get_text(separator=' ', strip=True)
            else:
                description = 'Описание не найдено'
            print(description)
            
                
            self.cursor.execute("""
                INSERT INTO info (url, title, price, photos, about_flat_jsonb, address, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
            """, (url, title, price, images_jsonb, about_flat_jsonb, address, description))
            self.conn.commit()

        except Exception as e:
            print(f"Ошибка при парсинге {url}: {str(e)}")
    
    
    
    def parse_all_ads_from_db(self, limit=10):
        try:
            self.cursor.execute("""
                SELECT url FROM url_to_ads 
                WHERE url NOT IN (SELECT url FROM info)
                LIMIT %s;
            """, (limit,))
            urls = self.cursor.fetchall()

            print(f"\nНачинаем обработку {len(urls)} объявлений из БД\n")

            for idx, (url,) in enumerate(urls, 1):
                print(f"\n[{idx}/{len(urls)}] Парсим: {url}")
                self.parse_single_ad(url)
                time.sleep(random.uniform(2, 4))  

        except Exception as e:
            print(f"Ошибка при получении ссылок из БД: {str(e)}")

In [22]:

def main():
    db_config = {
        'dbname': 'avito',
        'user': 'postgres',
        'password': 'Artem_17082003',
        'host': 'localhost',
        'port': '5432'
    }
    
    base_url = 'https://www.avito.ru/all/kvartiry/prodam-ASgBAgICAUSSA8YQ?...'

    # ЭТАП 1: Сбор ссылок
    url_collector = UrlCollector(base_url, db_config)
    try:
        print("\nЗапуск сбора ссылок на объявления")
        url_collector.process_all_ranges()
        print("\nСбор ссылок завершён успешно")
    except Exception as e:
        print(f"\nОшибка на этапе сбора ссылок: {e}")
    finally:
        url_collector.browser.quit()
        print("\nБраузер UrlCollector закрыт")

    # ЭТАП 2: Парсинг самих объявлений
    ad_parser = AdParser(db_config)
    try:
        print("\nЗапуск парсинга объявлений")
        ad_parser.parse_all_ads_from_db(limit=100)  # Например, сразу 100 объявлений
        print("\nПарсинг объявлений завершён")
    except Exception as e:
        print(f"\nОшибка на этапе парсинга объявлений: {e}")
    finally:
        ad_parser.browser.quit()
        print("\nБраузер AdParser закрыт")

if __name__ == '__main__':
    main()


Запуск сбора ссылок на объявления

=== Обработка диапазона 1000-450000 ===
Установлен диапазон: 1000 - 450000 руб.
1. https://www.avito.ru/ugolnye_kopi/kvartiry/dolya_v_2-k._kvartire_551_m_55_et._4671594772?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJlb3FjaTdoNXNLanNMa00zIjt9xhWGiT8AAAA
2. https://www.avito.ru/totma/kvartiry/auktsion_2-k._kvartira_43_m_12_et._7337709011?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJlb3FjaTdoNXNLanNMa00zIjt9xhWGiT8AAAA
3. https://www.avito.ru/kamchatskiy_kray_klyuchi/kvartiry/dolya_v_1-k._kvartire_401_m_22_et._4415219587?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJlb3FjaTdoNXNLanNMa00zIjt9xhWGiT8AAAA
4. https://www.avito.ru/pestravka/kvartiry/2-k._kvartira_462_m_12_et._4489315522?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJlb3FjaTdoNXNLanNMa00zIjt9xhWGiT8AAAA
5. https://www.avito.ru/va

TypeError: AdParser.__init__() missing 1 required positional argument: 'db_config'